In [1]:
import pandas as pd
import numpy as np
import random
from scipy.stats import bernoulli
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from scipy.stats import percentileofscore

import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.optim import AdamW

from transformers import (T5ForConditionalGeneration,
                          T5Tokenizer)
from datasets import Dataset

device = torch.device("cuda:1")
random.seed(42)

BATCH_SIZE = 16
EPOCHS     = 10
TAU        = 0.0
DIAG_PROP  = 0.8

def sample_disparity(actual_treatment, race, tau, diag_prop):
    race = race.apply(lambda x: 0 if x=='WHITE' else 1)
    bias = race*actual_treatment*tau
    prob = actual_treatment.apply(lambda x: x*diag_prop) - bias
    # temp = temp.apply(lambda x: 1 if x>1 else 0 if x<0 else x)
    return bernoulli.rvs(prob)

class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim=1):
        super(Classifier, self).__init__()
        
        self.hidden_layer1 = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.LeakyReLU(0.2)
        )

        self.hidden_layer2 = nn.Sequential(
            nn.Linear(128, 32),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )

        self.hidden_layer3 = nn.Sequential(
            nn.Linear(32, output_dim),
            nn.Sigmoid()
        )

    def forward(self, x, labels=None):
        output = self.hidden_layer1(x)
        output = self.hidden_layer2(output)
        output = self.hidden_layer3(output)
        return output

In [2]:
df = pd.read_csv('data/preprocessed.csv', lineterminator='\n')
df = df.sort_values(by=['SUBJECT_ID','HADM_ID','CHARTDATE'])\
                .groupby(['SUBJECT_ID','HADM_ID'])\
                .head(1)
                
idx_white = random.sample(list(df.loc[df.ETHNICITY=='WHITE'].index), 
              df.loc[df.ETHNICITY=='BLACK'].shape[0])
df = pd.concat([df.loc[idx_white], 
                df.loc[df.ETHNICITY=='BLACK']]).reset_index(drop=True)

df = df[['SUBJECT_ID','ETHNICITY','DIAGNOSIS','TEXT','apsiii']].reset_index(drop=True)
# df['apsiii_norm'] = (df['apsiii']-min(df['apsiii']))/(max(df['apsiii'])-min(df['apsiii']))
df['apsiii_norm'] = list(map(lambda x: percentileofscore(df['apsiii'], x, 'mean')/100, 
                            df['apsiii']))
df['actual_treatment'] = bernoulli.rvs(df['apsiii_norm'])
df['given_treatment'] = sample_disparity(df['actual_treatment'], df['ETHNICITY'], TAU, DIAG_PROP)

In [3]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, _ = RandomOverSampler(random_state=42).fit_resample(df_train, df_train['given_treatment'])
df_test, _ =  RandomOverSampler(random_state=42).fit_resample(df_test,  df_test['given_treatment'])

dataset_train = Dataset.from_pandas(df_train, split='train')
dataset_test  = Dataset.from_pandas(df_test,  split='test')

dataloader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
dataloader_test  = DataLoader(dataset_test,  batch_size=BATCH_SIZE, shuffle=True)

In [4]:
df_stats = df.groupby(['ETHNICITY','actual_treatment']).aggregate({'SUBJECT_ID':'count',
                                                        'given_treatment':'sum'})
df_stats['prop'] = df_stats['given_treatment']/df_stats['SUBJECT_ID']
print(f"Actual Treatment: {np.mean(df['actual_treatment'])}, Given Treatment: {np.mean(df['given_treatment'])}")
df_stats


Actual Treatment: 0.4972369053339741, Given Treatment: 0.39968765016818836


SUBJECT_ID  given_treatment      prop
ETHNICITY actual_treatment                                       
BLACK     0                       2065                0  0.000000
          1                       2097             1682  0.802098
WHITE     0                       2120                0  0.000000
          1                       2042             1645  0.805583

In [5]:
def encode(examples, tokenizer):
    inputs = examples['TEXT']  
    labels = nn.functional.one_hot(examples['given_treatment'], 
                                   num_classes=2)
    tokenized_inputs = tokenizer(inputs,
                                 return_tensors='pt',
                                 max_length=512,
                                 truncation=True,
                                 padding=True)
    model_inputs = {}
    model_inputs['input_ids']      = tokenized_inputs['input_ids']
    model_inputs['attention_mask'] = tokenized_inputs['attention_mask']
    return model_inputs, labels.float()


In [6]:
# Load debiased GAN, only keep the encoder
model = T5ForConditionalGeneration.from_pretrained("results/gan/model-1-final")
model_enc = model.encoder
del model
model_enc.to(device)

tokenizer = T5Tokenizer.from_pretrained("t5-small",
                                            output_scores=True,
                                            output_hidden_states=True,
                                            model_max_length=512)
# Define the models and optimizer
classifier = Classifier(input_dim=512, output_dim=2).to(device)
classifier_optimizer = AdamW(classifier.parameters(), 
                          lr=5e-3)

# Define loss
criterion = nn.BCELoss()

In [7]:
def train_classifier(batch, labels):
    classifier.train()
    classifier_optimizer.zero_grad()
    
    x = model_enc(**batch)                                       # Get embeddings from encoder
    x = x.last_hidden_state.sum(axis=1).detach()                 # Sum across emb_dim, detach

    pred = classifier(x)                                         # Make prediction
    loss = criterion(pred, labels)
    loss.backward()
    classifier_optimizer.step()
    return loss

def eval_classifier(batch, labels):
    classifier.eval()
    with torch.no_grad():
        x = model_enc(**batch)                                       # Get embeddings from encoder
        x = x.last_hidden_state.sum(axis=1).detach()                 # Sum across emb_dim, detach

        pred = classifier(x)                                         # Make prediction
        loss = criterion(pred, labels)
        return loss

In [8]:
def train(ep):
    train_loss, test_loss, steps = 0, 0, 0
    for batch in dataloader_train:
        steps += 1
        batch, labels           = encode(batch, tokenizer)           # Tokenize and obtain labels
        batch['input_ids']      = batch['input_ids'].to(device)      # Send to GPU
        batch['attention_mask'] = batch['attention_mask'].to(device)
        labels                  = labels.to(device)
        train_loss += float(train_classifier(batch, labels).detach().cpu())
        # if steps % 500 == 0:
        #     torch.save(classifier.state_dict(), f"results/gan/classifier-{ep}-{steps}-{TAU}-{DIAG_PROP}")

    for batch in dataloader_test:
        batch, labels           = encode(batch, tokenizer)           # Tokenize and obtain labels
        batch['input_ids']      = batch['input_ids'].to(device)      # Send to GPU
        batch['attention_mask'] = batch['attention_mask'].to(device)
        labels                  = labels.to(device)
        test_loss += float(eval_classifier(batch, labels).detach().cpu())
    
    train_loss /= len(dataloader_train)
    test_loss  /= len(dataloader_test)
    print(f"Epoch {ep}: {train_loss}, {test_loss}")
    return train_loss, test_loss

In [9]:
best_test = np.inf
for ep in range(EPOCHS):
    train_loss, test_loss = train(ep)
    if test_loss < best_test:
        best_test = test_loss
        torch.save(classifier.state_dict(), f"results/gan/classifier-{TAU}-{DIAG_PROP}-final")

Epoch 0: 0.7336700940467268, 0.6799268079554941
Epoch 1: 0.6885591240293051, 0.7273455196478236
Epoch 2: 0.6992151338651956, 0.7373737711606063
Epoch 3: 0.6902955365946972, 0.6746184586539982
Epoch 4: 0.6940793058120582, 0.7025298823521832
Epoch 5: 0.7068158060791023, 0.791305778533455
Epoch 6: 0.6874252450633719, 0.7867463446977571
Epoch 7: 0.6893449221030775, 0.6731536571435103
Epoch 8: 0.6954991177741783, 0.7164139583354859
Epoch 9: 0.6942660197555779, 0.7003768654320184


### Prediction/Evaluation

In [23]:
PNEUMONIA_KEYS = ['PNEUMONIA','PMEUMONIA','PNEUMOMIA',
                  'PNEUMONI','PNAUMONIA','PNEMONIA',
                  'PNEUMNOIA','PNEUMONIN','PNEUMONNIA']
FEVER_KEYS = ['FEVER','FEER']

df_filter = df.loc[df['DIAGNOSIS'].apply(lambda s: any([k in s for k in PNEUMONIA_KEYS]))]
# df_filter = df.loc[df['DIAGNOSIS'].apply(lambda s: any([k in s for k in FEVER_KEYS]))]

dataset_filter  = Dataset.from_pandas(df_filter,  split='filter')
dataloader_filter = DataLoader(dataset_filter, batch_size=BATCH_SIZE, shuffle=True)

In [24]:
race_lst, actual_lst, pred_lst, apsiii_lst = [], [], [], []
for idx, batch in enumerate(dataloader_filter):
    # print(f"Predicting: Batch {idx}")
    
    # Obtain actual treatment and race features
    actual_treatment        = nn.functional.one_hot(batch['actual_treatment'], 
                                                    num_classes=2).float()
    race                    = torch.tensor(list(map(lambda x: 1*(x=='BLACK'), 
                                                    batch['ETHNICITY'])))
    apsiii = batch['apsiii']
    
    batch, labels           = encode(batch, tokenizer)           # Tokenize and obtain labels
    batch['input_ids']      = batch['input_ids'].to(device)      # Send to GPU
    batch['attention_mask'] = batch['attention_mask'].to(device)
    labels                  = labels.to(device)
    
    x = model_enc(**batch)                                       # Get embeddings from encoder
    x = x.last_hidden_state.sum(axis=1).detach()                 # Sum across emb_dim, detach
    pred = classifier(x).cpu().detach()                          # Make prediction
    
    race_lst.append(race)
    actual_lst.append(actual_treatment)
    pred_lst.append(pred)
    apsiii_lst.append(apsiii)

race_lst   = torch.concat(race_lst)
actual_lst = torch.concat(actual_lst)
pred_lst   = torch.concat(pred_lst) 
apsiii_lst = torch.concat(apsiii_lst)

idx_0 = torch.where(race_lst==0)[0]
idx_1 = torch.where(race_lst==1)[0]

acc_1 = float(torch.mean(1.0*(actual_lst[idx_1].argmax(dim=1)==pred_lst[idx_1].argmax(dim=1))))
acc_0 = float(torch.mean(1.0*(actual_lst[idx_0].argmax(dim=1)==pred_lst[idx_0].argmax(dim=1))))
roc_0 = roc_auc_score(y_true  = actual_lst[idx_0].numpy(),
              y_score = pred_lst[idx_0].numpy())
roc_1 = roc_auc_score(y_true  = actual_lst[idx_1].numpy(),
              y_score = pred_lst[idx_1].numpy())
print(f"White: {acc_0} (Acc) {roc_0} ROC-AUC, Black: {acc_1} (Acc) {roc_1} ROC-AUC")

White: 0.6415094137191772 (Acc) 0.5572335513659099 ROC-AUC, Black: 0.6107784509658813 (Acc) 0.5957313245448839 ROC-AUC


In [29]:
df_analysis

,race,actual,pred_not_prescribe,pred_prescribe,apsiii,prescribe,correct
0,0,1,0.402421,0.597613,39,1,1
1,1,1,0.372129,0.627998,122,1,1
2,1,1,0.333899,0.666083,28,1,1
3,1,1,0.597312,0.402755,59,0,0
4,0,1,0.355063,0.645152,38,1,1
...,...,...,...,...,...,...,...
321,1,0,0.275927,0.724148,30,1,0
322,0,0,0.323179,0.676949,40,1,0
323,0,0,0.411530,0.588668,35,1,0
324,1,1,0.364907,0.635409,84,1,1


In [28]:
df_analysis = pd.DataFrame({'race':race_lst.numpy(),
              'actual':np.argmax(actual_lst.numpy(), axis=1),
              'pred_not_prescribe':pred_lst[:,0].numpy(),
              'pred_prescribe':pred_lst[:,1].numpy(),
              'apsiii':apsiii_lst.numpy()})
df_analysis['prescribe'] = 1*(df_analysis['pred_prescribe'] > df_analysis['pred_not_prescribe'])
df_analysis['correct'] = 1*(df_analysis['prescribe']==df_analysis['actual'])

for i in range(8):
    if i<=5:
        temp = df_analysis.loc[(df_analysis.apsiii >= 20*i)&\
                            (df_analysis.apsiii < 20*(i+1))]\
                                .groupby('race')\
                                .aggregate({'correct':'mean', 'prescribe':'mean', 'apsiii':'count'})
    else:
        temp = df_analysis.loc[(df_analysis.apsiii >= 20*i)]\
                                .groupby('race')\
                                .aggregate({'correct':'mean', 'prescribe':'mean', 'apsiii':'count'})
    print(f"Accuracy (Black): {round(100*temp['correct'][1],3)} ({round(100*temp['prescribe'][1],3)}% prescribed, {temp['apsiii'][1]} patients), \t Accuracy (White): {round(100*temp['correct'][0],3)} ({round(100*temp['prescribe'][0],3)}% prescribed, {temp['apsiii'][0]} patients)")


Accuracy (Black): 33.333 (33.333% prescribed, 3 patients), 	 Accuracy (White): 50.0 (50.0% prescribed, 2 patients)
Accuracy (Black): 42.593 (72.222% prescribed, 54 patients), 	 Accuracy (White): 50.0 (73.684% prescribed, 38 patients)
Accuracy (Black): 70.0 (74.286% prescribed, 70 patients), 	 Accuracy (White): 62.857 (82.857% prescribed, 70 patients)
Accuracy (Black): 65.217 (69.565% prescribed, 23 patients), 	 Accuracy (White): 70.968 (74.194% prescribed, 31 patients)
Accuracy (Black): 76.923 (69.231% prescribed, 13 patients), 	 Accuracy (White): 83.333 (83.333% prescribed, 12 patients)
Accuracy (Black): 100.0 (100.0% prescribed, 1 patients), 	 Accuracy (White): 100.0 (100.0% prescribed, 5 patients)
Accuracy (Black): 100.0 (100.0% prescribed, 3 patients), 	 Accuracy (White): 100.0 (100.0% prescribed, 1 patients)


KeyError: 1